# Identifying Phase Transitions

In this notebook, we will use numerous methods to characterize the phase behavior of our polygons.
As a first step, we will want to identify whether, for a particular combination of parameters, our system undergoes some sort of phase transition once the simulation runs for long enough.
We will perform various analyses to show how we can identify this phase transition behavior.
Once we have identified that this occurs for a some pressure for a given shape, we will look at multiple data points in concert to identify the critical point, the pressure value above which all systems of a given type of polygon would be expected to undergo this phase transition.

In [ ]:
import signac

project = signac.get_project()

In [ ]:
import sys
from ipywidgets import interact

JOB = None

schema = project.detect_schema()
ns = schema['n'][int]
betaPs = schema['betaP'][float]
seeds = schema['seed'][int]

@interact(n=ns, betaP=betaPs, seed=seeds)
def select(n=5, betaP=13.2, seed=0):
    global JOB
    jobs = project.find_jobs(dict(n=n, betaP=betaP, seed=seed))
    if len(jobs) == 0:
        print("No jobs found for this selection.", file=sys.stderr)
    elif len(jobs) > 1:
        print("Multiple jobs matched this selection.", file=sys.stderr)
    else:
        JOB = list(jobs)[0]
        print("Selected", JOB)

As a first step, we plot the hexatic order parameter over time.
Generally speaking, 2D systems of particles are expected to exhibit a [hexatic phase](https://en.wikipedia.org/wiki/Hexatic_phase), in which the system has 6-fold translational ordering.
The [k-atic order parameter](https://freud.readthedocs.io/en/stable/order.html#freud.order.HexOrderParameter) measures *k-fold* order in a system of particles by calculating the angles between a particle and its neighbors.
In this case, over the course of the simulation we have calculated and stored the 6-atic, or **hexatic**, order parameter, which measures the 6-fold order.
In the plots below, we see how the value of the order parameter increases over the course of the simulation as the system becomes ordered.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import gsd.hoomd
%matplotlib inline

with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
    N = traj[-1].particles.N
# alternately, use a custom log operation to log phi directly

log = np.genfromtxt(fname=JOB.fn('log.dat'), names=True)
psi = np.load(JOB.fn('order.npz'))

fig, ax = plt.subplots(figsize=(4, 2.2), dpi=140)
ax2 = ax.twinx()

ax.plot(log['timestep'], N * JOB.doc.poly_area / log['volume'])
ax.set_xlabel('time step')
ax.set_ylabel('Packing Fraction')

ax2.plot(psi['steps'], np.absolute(psi['psi'].mean(axis=1)), color='red')
ax2.set_ylabel('Hexatic Order Parameter ($\psi$)', color='red')

plt.show()

From this plot, we see that 6-fold ordering is eventually arising over the course of this simulation.
To get a more concrete picture of what this looks like, we can perform a number of different analyses.
As a first step, we can visualize the system directly.
In the plot below, the shapes are colored by the value of the hexatic order parameter.
Note that instead of using the stored values, we are now using [`freud`](https://freud.readthedocs.io/en/stable/order.html#freud.order.HexOrderParameter) to calculate the order parameter on the fly each time we visualize a new frame.

In [ ]:
import freud
from draw_utils import quat2ang, draw_config, draw_pmft, draw_voronoi

with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
    num_frames = len(traj)-1 

In [ ]:
@interact(frame=(0, num_frames))
def frame_demo(frame=num_frames):
    fix, ax = plt.subplots(1, 1, figsize=(8, 8))
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
        
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        hop = freud.order.HexOrderParameter(rmax=1.2, k=6)
        hop.compute(box, frame.particles.position)
        
        draw_config(fig, ax, box, frame.particles.position, quat2ang(frame.particles.orientation), hop.psi, JOB.sp.n)

Evidently, we see that while early on in the simulation the value of the order parameter is highly variable throughout the system, towards the end the hexatic order parameter is similar for nearly all of the particles (as shown by the relative homogeneity of the color scheme).
One way we can try to see this more clearly is using the [radial distribution function (RDF)](https://en.wikipedia.org/wiki/Radial_distribution_function), which measures the system density as a function of distance from a particle.
The RDF is measured by calculating the average number of particles at a given distance from each particle and then averaging that measure over all particles.
In a perfect crystal, the RDF should be just a set of [delta functions](https://en.wikipedia.org/wiki/Delta_function) since all particles are located at precise fixed distances from one another.
In systems of particles like the ones we are simulating, we instead expect to see smoother sets of peaks.

Below, we calculate the RDF by binning all particles using `freud` (see [the documentation](https://freud.readthedocs.io/en/stable/density.html#freud.density.RDF) for more information).

In [ ]:
@interact(frame=(0, num_frames))
def rdf_demo(frame=num_frames):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        rdf = freud.density.RDF(np.sqrt(box.Lx**2 + box.Ly**2)/5, box.Lx/1000)
        rdf.compute(box, frame.particles.position)

    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.plot(rdf.R, rdf.RDF)

From the plots above, we can see that while the initial frames show a generally uniform density after the nearest neighbor peak, over time the RDF exhibits more regular peaks.
However, as a function of distance alone, the RDF is clearly a limited way of characterizing our system, which we know should exhibit some ordering that depends on the angle between particles.

To get a more informative picture, we can look at the potential of mean force and torque (PMFT).
The PMFT is a generalization of the classical [potential of mean force](https://en.wikipedia.org/wiki/Potential_of_mean_force) (PMF), which measures the average potential energy surface about each particle in the system as a function of distance.
The PMF can be calculated directly as $w(r) = -\beta \log(g(r))$ (where $g(r)$ is the radial distribution function).

The PMFT takes this one step further, looking at the potential energy surface as a function of both distance and angle.
As a result, the PMFT can capture the differences in a potential energy surface induced by, for instance, the shape of a particle.
In practice, the PMFT is calculated by [binning space and counting particles](https://freud.readthedocs.io/en/stable/pmft.html#freud.pmft.PMFTXY2D).
We plot the results below.

In [ ]:
@interact(frame=(0, num_frames))
def pmft_demo(frame=num_frames):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        pmft = freud.pmft.PMFTXY2D(4, 4, 300, 300)
        pmft.compute(box, frame.particles.position, quat2ang(frame.particles.orientation))

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    draw_pmft(fig, ax, pmft, JOB.sp.n)

Looking at early frames of our simulation, we see that the PMFT looks almost spherically uniform.
In fact, in this case the PMFT shows the same information as the RDF.
There is a low energy ring immediately surrounding the central particle, and past that we see a mostly uniform potential energy surface at all distances.
In later frames, however, we start to see more features.
In particular, the PMFT shows smaller energy wells that represent the later peaks in the RDF that appeared at later frames.
With the PMFT, however, we get some additional information: instead of being circular, each subsequent ring has a shape commensurate to our actual particle shape.
This shape reflects the fact that not only do particles tend to sit as specific distances from one another, the optimal distance is also a function of the angle between the particles.

For an even more visually obvious metric of this anistropy, we can look at the [Voronoi diagram](https://en.wikipedia.org/wiki/Voronoi_diagram) of the system.
Using a fixed set of reference points, the Voronoi diagram creates a partitioning of space where every point is associated with the closest reference point.
In a system of perfectly ordered polygons, we would expect the Voronoi diagram to divide space into a set of identical polygons corresponding exactly to the actual polygons in the system.
Anywhere there are defects in the system, however, we would expect the Voronoi diagram to show some disorder.
To demonstrate this, we plot the Voronoi diagrams below, with each Voronoi polygon colored by the number of sides.

In [ ]:
@interact(frame=(0, num_frames))
def voronoi_demo(frame=num_frames):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        voronoi = freud.voronoi.Voronoi(box, np.sqrt(box.Lx**2 + box.Ly**2)/5)
        voronoi.compute(frame.particles.position, box)

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    draw_voronoi(fig, ax, box, voronoi.polytopes)

As we see, early on in the simulation the Voronoi polygons are polydisperse, as there is a relatively inhomogeneous distribution of our real polygons throughout the system.
As the system becomes ordered, the Voronoi diagram also reflects this order.
In the final frame, almost all of the Voronoi polygons have the same number of sides as our original polygons, and the areas where this is not true are clear signs of defects.

Finally, we return to our original problem: finding the pressure at which the system undergoes a phase transition.
We plot the average value of the hexatic order parameter (in the final $N$ frames of our simulation) as a function of pressure, and we look for a pressure at which we see a sudden jump in the order parameter.

In [ ]:
@interact(n=project.detect_schema()['n'][int], num_frames=(1, 10))
def transition_demo(n, num_frames=5):
    avg_hop = {}
    for betaP, group in project.find_jobs({"n": n}).groupby('betaP'):
        hops = []
        for job in group:
            with gsd.hoomd.open(job.fn('trajectory.gsd')) as traj:
                for frame in traj[-num_frames:]:
                    box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
                    hop = freud.order.HexOrderParameter(rmax=1.2, k=6)
                    hop.compute(box, frame.particles.position)
                    hops.append(np.absolute(np.mean(hop.psi)))
        avg_hop[betaP] = np.mean(hops)
    fig, ax = plt.subplots(1, 1)
    ax.plot(avg_hop.keys(), avg_hop.values())